
# Sparse Vector Retrieval (Récupération par vecteurs creux)

Ce notebook est un **exemple pédagogique** pour introduire la récupération d'information (IR) avec des représentations **sparse** (creuses).  
On explore trois approches classiques :

1. **Bag of Words / CountVectorizer** — fréquence brute des mots (sac de mots).  
2. **TF‑IDF (Term Frequency – Inverse Document Frequency)** — pondère les termes fréquents dans un document mais rares dans la collection.  
3. **BM25** — une famille de scores probabilistes très utilisée dans les moteurs de recherche.  

Chaque sous‑section inclut : une **explication** + un **exemple de code** pour indexer une mini‑collection et retrouver les documents les plus pertinents pour une requête.

> *Note « Hugging Face » : dans cette section « sparse », nous n'avons pas besoin de modèles HF. Les parties denses (Section 2) tireront parti d'`transformers`/`sentence-transformers`. Ici, on se concentre sur les méthodes classiques ne nécessitant pas de réseau de neurones.*



## Préparation
On définit une petite collection de documents (un mini‑corpus) pour l'exercice et quelques requêtes.  
On utilisera **scikit‑learn** pour *CountVectorizer* et *TfidfVectorizer*, et une **implémentation BM25 maison** pour éviter toute dépendance externe.


In [14]:

# Imports de base
from typing import List, Tuple
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Un mini-corpus jouet (FR + un peu d'EN pour montrer la robustesse)
corpus = [
    # --- Concepts généraux & définitions ---
    "La recherche d'information (IR) vise à retrouver des documents pertinents pour un besoin utilisateur exprimé sous forme de requête.",
    "Un pipeline RI comporte souvent : prétraitement, indexation, pondération, scoring, puis évaluation des résultats.",
    "Le modèle sac de mots ignore l'ordre des termes et représente un document par la fréquence des mots.",
    "TF-IDF pondère fort les mots fréquents dans un document mais rares dans le corpus ; TF peut être brut, log-normalisé ou binaire.",
    "Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.",
    "BM25+ et BM25L sont des variantes qui ajustent la normalisation de longueur et l'offset des faibles fréquences.",
    "Les embeddings denses encodent le sens dans un espace vectoriel continu, utiles pour la recherche sémantique.",
    "La recherche vectorielle peut être clairsemée (sparse) ou dense selon la représentation.",
    "RAG (Retrieval-Augmented Generation) combine récupération de passages et modèles génératifs pour produire des réponses sourcées.",
    "La pertinence peut être binaire ou graduée (gains), ce qui change la métrique optimale.",

    # --- Prétraitement & normalisation ---
    "La normalisation inclut les accents, la casse, la tokenisation, le stemming et la lemmatisation.",
    "Les mots vides (stopwords) comme 'le', 'la', 'de' sont souvent supprimés pour réduire le bruit.",
    "La lemmatisation française nécessite de gérer les accords et les formes verbales composées.",
    "Le stemming de Porter tronque les suffixes, au risque d'ambiguïtés (ex. 'nation' et 'national').",
    "Les fautes de frappe et variantes orthographiques (ex. 'recherhe', 'recherche') pénalisent les méthodes exactes.",
    "Les entités nommées (NER) aident à conserver des unités sémantiques comme 'OpenAI' ou 'Europe/Paris'.",

    # --- Outils & bibliothèques ---
    "Scikit-learn fournit CountVectorizer et TfidfVectorizer pour les représentations clairsemées.",
    "Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.",
    "FAISS, HNSW (hnswlib), Annoy et ScaNN accélèrent la recherche de plus proches voisins approximatifs (ANN).",
    "Sentence-BERT (SBERT), E5, MPNet et ColBERT sont populaires pour l'indexation dense ou late interaction.",
    "Elasticsearch et OpenSearch implémentent BM25 par défaut et supportent la recherche hybride sparse+dense.",

    # --- Évaluation ---
    "L'évaluation compare précision, rappel, F1, MAP, MRR, nDCG@k et taux de couverture des passages.",
    "MRR mesure l'inverse du rang de la première réponse correcte ; nDCG mesure des gains gradués.",
    "Les jeux de données TREC, MS MARCO et BEIR sont des bancs d'essai courants.",
    "La validation croisée et le split train/dev/test évitent le surapprentissage sur le corpus d'évaluation.",
    "Le jugement de pertinence peut être humain, heuristique ou obtenu via pooling multi-systèmes.",

    # --- Multilingue & domaines ---
    "Les requêtes peuvent être multilingues ; les modèles multilingues mappent plusieurs langues dans le même espace.",
    "Dans le domaine médical, la précision terminologique est cruciale (ex. 'hypertension artérielle').",
    "En droit, les synonymes et références croisées (articles, alinéas) créent de la variance lexicale.",
    "Les recettes de cuisine génèrent des listes d'ingrédients qui favorisent la pondération TF simple.",
    "La recherche académique bénéficie d'expressions multi-mots comme 'régression logistique' ou 'réseau de neurones'.",

    # --- Bruit, longueur et structure ---
    "Certains documents sont très courts: 'BM25 expliqué rapidement'.",
    "D'autres sont longs et contiennent des sections, des listes, des liens et des tableaux qui influencent la tokenisation.",
    "URLs et hashtags (#IR, #NLP) perturbent parfois la segmentation en tokens.",
    "Les citations de code `pip install faiss-gpu` ou `from sklearn.feature_extraction.text import TfidfVectorizer` doivent être préservées.",

    # --- Recherche hybride & reranking ---
    "La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.",
    "Le reranking par un cross-encoder (ex. monoT5, cross-encoder-MiniLM) affine les top-k candidats.",
    "La réduction de dimension (PCA) ou la quantification des vecteurs (PQ, IVFPQ) accélère l'ANN au prix d'une légère perte de rappel.",
    "Le champ 'title' a souvent plus de poids que 'body' dans les moteurs de recherche.",
    "La désambiguïsation des acronymes (ex. 'IR' = 'information retrieval' ou 'imagerie par résonance') dépend du contexte.",

    # --- Cas concrets & exemples mixtes ---
    "Le sac de mots (bag of words) est simple et robuste mais ne capture pas la sémantique profonde.",
    "TF–IDF (avec tiret demi-cadratin) est équivalent à TF-IDF (avec tiret standard) pour la plupart des implémentations.",
    "Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.",
    "Une requête négative comme 'BM25 sans normalisation de longueur' nécessite des opérateurs booléens.",
    "La recherche de vecteurs denses peut tolérer les fautes: 'embeding denses' correspond souvent à 'embeddings denses'.",
    "Vector search peut être sparse or dense depending on the representation (mélange FR/EN).",
    "L'alignement d'espaces multilingues permet 'apprentissage' ≈ 'learning' ≈ 'aprendizaje'.",
    "Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.",
    "Le choix des stop words en français diffère de l'anglais (ex. 'au', 'aux', 'des', 'du').",
    "Le pré-traitement influence fortement les résultats, surtout avec de petites requêtes.",
    "La normalisation Unicode (NFC/NFKD) affecte le traitement des diacritiques (é/ê/e).",
    "En RAG, l'ordre : retrieve → read → generate ; la qualité du retrieve détermine souvent 80% du succès perçu.",
    "Une indexation incrémentale maintient la fraîcheur des résultats sans reconstruire tout l'index."
]

queries = [
    # Requêtes ciblées IR
    "comment fonctionne BM25 (k1, b) pour le ranking ?",
    "différence entre sac de mots et TF-IDF",
    "CountVectorizer vs TfidfVectorizer scikit-learn",
    "bm25 ranking vs dense retrieval",
    "quels avantages des embeddings denses pour la sémantique ?",

    # Prétraitement & robustesse
    "liste de stopwords en français et impact sur la précision",
    "lemmatisation vs stemming pour le français",
    "normalisation des accents et diacritiques",
    "gerer les fautes de frappe en recherche d'information",
    "traiter les hashtags et urls dans les documents",

    # Évaluation
    "comparer MAP, MRR et nDCG@10",
    "datasets d'évaluation: TREC, MS MARCO, BEIR",
    "pooling et jugement de pertinence gradué",
    "quelle métrique pour la pertinence multi-niveaux ?",

    # Multilingue & domaines
    "recherche multilingue avec SBERT ou E5",
    "RAG pour questions médicales",
    "désambiguïser l'acronyme IR selon le contexte",
    "recherche juridique: articles et alinéas",
    "recettes: pondération TF simple vs BM25",

    # ANN & systèmes
    "HNSW vs FAISS vs Annoy pour ANN",
    "hybrid search: combiner BM25 et embeddings",
    "cross-encoder pour reranking des top-k",
    "quantification des vecteurs (IVFPQ) et rappels",
    "pondérer le champ title plus que body",

    # Requêtes bruitées / mixtes / typos
    "tf–idf (variante typographique) explication",
    "embeding denses (typo) utilité",
    "Okapi-BM25 formule",
    "vector search sparse or dense",
    "BM25 sans normalisation de longueur"
]
print(f"Documents: {len(corpus)}  |  Requêtes: {queries}")


Documents: 53  |  Requêtes: ['comment fonctionne BM25 (k1, b) pour le ranking ?', 'différence entre sac de mots et TF-IDF', 'CountVectorizer vs TfidfVectorizer scikit-learn', 'bm25 ranking vs dense retrieval', 'quels avantages des embeddings denses pour la sémantique ?', 'liste de stopwords en français et impact sur la précision', 'lemmatisation vs stemming pour le français', 'normalisation des accents et diacritiques', "gerer les fautes de frappe en recherche d'information", 'traiter les hashtags et urls dans les documents', 'comparer MAP, MRR et nDCG@10', "datasets d'évaluation: TREC, MS MARCO, BEIR", 'pooling et jugement de pertinence gradué', 'quelle métrique pour la pertinence multi-niveaux ?', 'recherche multilingue avec SBERT ou E5', 'RAG pour questions médicales', "désambiguïser l'acronyme IR selon le contexte", 'recherche juridique: articles et alinéas', 'recettes: pondération TF simple vs BM25', 'HNSW vs FAISS vs Annoy pour ANN', 'hybrid search: combiner BM25 et embeddings', 


### Fonction utilitaire d'affichage
Une petite fonction pour afficher les **Top‑k** documents retournés par chaque méthode avec leur score.


In [15]:

def show_results(query: str, doc_ids: List[int], scores: np.ndarray, top_k: int = 5):
    print(f"\nQuery: {query!r}")
    print("-" * 80)
    idx_sorted = np.argsort(scores)[::-1][:top_k]
    for rank, i in enumerate(idx_sorted, start=1):
        print(f"[{rank:>2}] score={scores[i]:.4f}  doc#{doc_ids[i]}  →  {corpus[doc_ids[i]]}")



## 1) Bag of Words / **CountVectorizer**

**Idée** : représenter chaque document par un vecteur de **comptes** de mots, sans tenir compte de l'ordre (modèle du **sac de mots**).  
- Avantages : simple, rapide, robuste sur de petits corpus.  
- Limites : pas de pondération par rareté, insensible aux synonymes et au contexte.

**Pipeline** :
1. *Tokenisation* + vocabulaire (par `CountVectorizer`).  
2. Encodage des documents et de la requête.  
3. Calcul de similarité (cosinus) puis tri décroissant des scores.


In [16]:

# Vectorisation BoW
count_vec = CountVectorizer(lowercase=True, stop_words=None)  # on garde simple
X_count = count_vec.fit_transform(corpus)

# Encodage d'une requête → même espace vectoriel
def bow_search(query: str) -> Tuple[np.ndarray, np.ndarray]:
    q_vec = count_vec.transform([query])
    # Similarité cosinus entre la requête et chaque document
    scores = cosine_similarity(q_vec, X_count).ravel()
    doc_ids = np.arange(X_count.shape[0])
    return doc_ids, scores

# Démo sur quelques requêtes
for q in queries:
    ids, sc = bow_search(q)
    show_results(q, ids, sc, top_k=5)



Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'
--------------------------------------------------------------------------------
[ 1] score=0.3254  doc#11  →  Les mots vides (stopwords) comme 'le', 'la', 'de' sont souvent supprimés pour réduire le bruit.
[ 2] score=0.3254  doc#42  →  Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 3] score=0.3078  doc#35  →  La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
[ 4] score=0.2860  doc#47  →  Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.
[ 5] score=0.2860  doc#24  →  La validation croisée et le split train/dev/test évitent le surapprentissage sur le corpus d'évaluation.

Query: 'différence entre sac de mots et TF-IDF'
--------------------------------------------------------------------------------
[ 1] score=0.4352  doc#2  →  Le modèle sac de mots ignore l'ordre des termes et représente un


## 2) **TF‑IDF** **(term frequency-inverse document frequency)**

**Idée** : pondérer les termes par la fréquence dans le document (**TF**) et la rareté dans la collection (**IDF**).  
Ainsi, un terme très courant dans tous les documents aura un poids plus faible qu'un terme rare.


On réutilise la similarité cosinus pour le ranking.


In [17]:

# Vectorisation TF-IDF
tfidf_vec = TfidfVectorizer(lowercase=True, stop_words=None, norm="l2")
X_tfidf = tfidf_vec.fit_transform(corpus)

def tfidf_search(query: str) -> Tuple[np.ndarray, np.ndarray]:
    q_vec = tfidf_vec.transform([query])
    scores = cosine_similarity(q_vec, X_tfidf).ravel()
    doc_ids = np.arange(X_tfidf.shape[0])
    return doc_ids, scores

for q in queries:
    ids, sc = tfidf_search(q)
    show_results(q, ids, sc, top_k=5)



Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'
--------------------------------------------------------------------------------
[ 1] score=0.4227  doc#47  →  Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.
[ 2] score=0.3281  doc#42  →  Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 3] score=0.2362  doc#35  →  La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
[ 4] score=0.2105  doc#4  →  Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.
[ 5] score=0.1755  doc#11  →  Les mots vides (stopwords) comme 'le', 'la', 'de' sont souvent supprimés pour réduire le bruit.

Query: 'différence entre sac de mots et TF-IDF'
--------------------------------------------------------------------------------
[ 1] score=0.3752  doc#2  →  Le modèle sac de mots ignore l'ordre des termes et r


## 3) **BM25** (Okapi)

**Idée** : une fonction de scoring inspirée des modèles probabilistes (Okapi BM25) qui corrige certains biais de fréquence et **normalise la longueur des documents**.

**Score BM25** pour un terme \(t\), un document \(d\) :
\[
\text{score}(d, q) = \sum_{t \in q} \text{idf}(t) \cdot
\frac{ tf_{t,d} (k_1 + 1)}{ tf_{t,d} + k_1\big(1 - b + b\,\frac{|d|}{\overline{|d|}}\big) }
\]
- \(k_1\) règle la saturation (souvent entre 1.2 et 2.0).  
- \(b\) contrôle la normalisation par longueur (souvent ~0.75).  
- \(\overline{|d|}\) est la longueur moyenne des documents.

On implémente une **version compacte** de BM25 pour notre mini‑corpus, sans dépendances externes.


In [18]:

import math
from collections import Counter, defaultdict

class BM25OkapiMini:
    def __init__(self, docs: List[str], tokenizer=str.split, k1: float = 1.5, b: float = 0.75):
        self.k1 = k1
        self.b = b
        self.tokenizer = tokenizer
        
        self.docs_tokens = [self.tokenizer(d.lower()) for d in docs]
        self.doc_lens = [len(toks) for toks in self.docs_tokens]
        self.avgdl = sum(self.doc_lens) / len(self.doc_lens)
        
       
        self.tf = [Counter(toks) for toks in self.docs_tokens]
        
        # document frequencies
        df = Counter()
        for counter in self.tf:
            for term in counter:
                df[term] += 1
        self.df = df
        self.N = len(docs)
        
    
        self.idf = {}
        for term, df_t in self.df.items():
            # idf BM25 classique (Robertson/Sparck Jones)
            self.idf[term] = math.log((self.N - df_t + 0.5) / (df_t + 0.5) + 1)
    
    def score(self, query: str) -> np.ndarray:
        q_terms = self.tokenizer(query.lower())
        scores = np.zeros(self.N, dtype=float)
        for i, tf_i in enumerate(self.tf):
            dl = self.doc_lens[i]
            denom_norm = self.k1 * (1 - self.b + self.b * dl / self.avgdl)
            s = 0.0
            for t in q_terms:
                if t not in tf_i:
                    continue
                idf_t = self.idf.get(t, 0.0)
                tf_td = tf_i[t]
                s += idf_t * ((tf_td * (self.k1 + 1)) / (tf_td + denom_norm))
            scores[i] = s
        return scores

bm25 = BM25OkapiMini(corpus)

def bm25_search(query: str) -> Tuple[np.ndarray, np.ndarray]:
    scores = bm25.score(query)
    doc_ids = np.arange(len(corpus))
    return doc_ids, scores

for q in queries:
    ids, sc = bm25_search(q)
    show_results(q, ids, sc, top_k=5)



Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'
--------------------------------------------------------------------------------
[ 1] score=5.8060  doc#42  →  Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 2] score=4.8605  doc#35  →  La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
[ 3] score=4.5743  doc#4  →  Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.
[ 4] score=2.9294  doc#11  →  Les mots vides (stopwords) comme 'le', 'la', 'de' sont souvent supprimés pour réduire le bruit.
[ 5] score=2.8425  doc#6  →  Les embeddings denses encodent le sens dans un espace vectoriel continu, utiles pour la recherche sémantique.

Query: 'différence entre sac de mots et TF-IDF'
--------------------------------------------------------------------------------
[ 1] score=6.4035  doc#2  →  Le modèle sac de mots ignore l'ordre des term


## 4) Comparaison rapide des méthodes

On lance les trois méthodes pour chaque requête et on compare visuellement les **Top‑3** résultats.  
Le but est d'observer des **différences de ranking** dues à la pondération (TF‑IDF) et à la normalisation de longueur (BM25).


In [19]:

def topk_indices(scores, k=3):
    return np.argsort(scores)[::-1][:k]

def compare_methods(query: str, k: int = 3):
    ids_bow, sc_bow = bow_search(query)
    ids_tfidf, sc_tfidf = tfidf_search(query)
    ids_bm25, sc_bm25 = bm25_search(query)
    
    top_bow = topk_indices(sc_bow, k)
    top_tfidf = topk_indices(sc_tfidf, k)
    top_bm25 = topk_indices(sc_bm25, k)
    
    print(f"\n=== Query: {query!r} ===")
    print("BoW/CountVectorizer:")
    for r, i in enumerate(top_bow, 1):
        print(f"  {r}. ({sc_bow[i]:.4f}) {corpus[i]}")
    print("TF-IDF:")
    for r, i in enumerate(top_tfidf, 1):
        print(f"  {r}. ({sc_tfidf[i]:.4f}) {corpus[i]}")
    print("BM25:")
    for r, i in enumerate(top_bm25, 1):
        print(f"  {r}. ({sc_bm25[i]:.4f}) {corpus[i]}")

for q in queries:
    compare_methods(q, k=3)



=== Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?' ===
BoW/CountVectorizer:
  1. (0.3254) Les mots vides (stopwords) comme 'le', 'la', 'de' sont souvent supprimés pour réduire le bruit.
  2. (0.3254) Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
  3. (0.3078) La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
TF-IDF:
  1. (0.4227) Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.
  2. (0.3281) Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
  3. (0.2362) La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
BM25:
  1. (5.8060) Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
  2. (4.8605) La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
  3. (4.5743) Okapi BM25 est un schéma


## 5) Notes pratiques & lien avec la suite (dense + Hugging Face)

- Sur de **grands corpus**, BM25 et TF‑IDF restent des **baselines solides** et étonnamment compétitives.  
- Pour des requêtes courtes, BM25 a souvent un avantage sur **BoW** car il normalise la longueur et pondère mieux les mots rares.  
- Ces méthodes ignorent le **sens** et les **synonymes** — d'où l'intérêt des **représentations denses** , souvent construites avec des modèles *Hugging Face* (p. ex. `sentence-transformers` basés sur BERT).  
